贝叶斯优化法的工作原理是构建一个函数的后验分布（高斯过程），它能最好地描述你想要优化的函数。如下图所示，随着观测数据数量的增加，后验分布会不断改善，算法也会更加确定参数空间中哪些区域值得探索，哪些不值得探索。



在反复迭代的过程中，算法会根据对目标函数的了解，平衡探索和利用的需要。每一步都会对已知样本（之前探索过的点）进行高斯过程拟合，并结合后验分布和探索策略（如 UCB（置信度上限）或 EI（预期改进））来确定下一个要探索的点（见下图）。

如何在 Python 中实现这些解决方案？
在 Python 中实现超参数调整有不同的方法，这取决于你选择的库和方法。scikit-learn 是一个流行的 Python 机器学习库，它提供了多个用于超参数调整的工具和类。其中最常见的有

GridSearchCV：该类在超参数网格上执行穷举搜索，使用交叉验证评估每个组合。

RandomizedSearchCV：该类对超参数分布进行随机搜索，抽取固定数量的候选参数，并使用交叉验证对其进行评估。

HalvingGridSearchCV和HalvingRandomSearchCV：这两个类与 GridSearchCV 和 RandomizedSearchCV 类似，但它们使用了连续减半技术，在每次迭代时根据候选数据的性能减少候选数据的数量。这可以加快搜索过程并降低计算成本。

另一个流行的 Python 机器学习库是 Neptune.ai，它为实验跟踪和超参数调整提供了一个平台。您可以通过安装 neptune-client 软件包并在 Neptune 网站上创建一个项目来使用它。

**Optuna "库在 Kaggle 竞赛的参赛者中拥有众多粉丝，它在自己的网站上做了如下介绍： Optuna 采用最先进的算法对超参数进行采样，并有效剪除不具潜力的试验，从而实现高效的超参数优化。此外，该链接还介绍了七种采样算法。**

**bayes_opt： 这是一个使用高斯过程实现贝叶斯优化算法的库。您可以通过安装 bayes_opt 软件包并创建一个带有目标函数和参数边界的 BayesianOptimization 对象来使用它。然后，你就可以调用 maximize 方法来找到最佳参数和得分。**

hyperopt： 这是一个实现各种超参数优化方法的库，例如树状结构帕尔森估计器（TPE）、自适应 TPE 和随机搜索。

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats
import plotly.figure_factory as ff
import plotly.express as px
pd.set_option('display.max_columns',100)

In [ ]:
df_train=pd.read_csv("C:/Users/15295/Desktop/Kaggle/用户交易行为预测/train.csv")
df_test=pd.read_csv("C:/Users/15295/Desktop/Kaggle/用户交易行为预测/test.csv")
df_sample=pd.read_csv("C:/Users/15295/Desktop/Kaggle/用户交易行为预测/sample_submission.csv")

In [ ]:
display(df_train.info())
df_train.describe()

In [ ]:
df_train.isnull().sum(),df_test.isnull().sum()

In [ ]:
train=df_train.copy()
test=df_test.copy()

target=train.pop('target')
target.value_counts().plot(kind='barh',figsize=(12,1),title='Target Count',color=['darkcyan','red'])
plt.gca().set_facecolor('lightgray')



In [ ]:
target.value_counts().plot(kind='pie',figsize=(4,4),title='Proportion of Target',autopct='%1.0f%%')

In [ ]:
features=train.columns.tolist()

In [ ]:
#随机选取一些特诊进行可视化分析

features_select=np.random.choice(features,size=28,replace=False).tolist()

In [ ]:
corr_matrix=df_train[features_select+['target']].corr().round(2)#加上target这个标签

fig=plt.figure(figsize=(12,12))
cmap=sns.color_palette('Blues')
sns.heatmap(corr_matrix,center=0,annot=False,cmap=cmap)
plt.show()

In [ ]:
# sns.set()
# plt.style.use('seaborn-whitegrid')
# _,axs=plt.subplots(7,4,figsize=(15,30),facecolor='lightyellow')

# for f,ax in zip(features_select,axs.ravel()):
#     ax.set_facecolor('lightcyan')
#     ax.hist(train[f],bins=100,color='red')
#     ax.set_title(f'Feature:{f}',fontsize=10)
    
# plt.suptitle('Histograms of the Randomly selected features',y=0.91,fontsize=14)
# plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve,auc

def roc_auc(true_list,pred_list,a,b):
    fpr,tpr,_=roc_curve(true_list,pred_list)#在刻画roc曲线的时候，我们应该使用标签列和预测的概率或者分数
    roc_auc=auc(fpr,tpr)
    
    print(f'\n>>>>> ROC_AUC: %0.6f <<<<<\n %roc_auc')
    
    sns.set()
    plt.style.use('seaborn-whitegrid')
    plt.figure(figsize=(a,b),facecolor='lightyellow')
    plt.gca().set_facecolor('lightblue')
    plt.plot(fpr,tpr,color='darkorange',lw=2,label='ROC curve')
    plt.plot([0,1],[0,1],color='navy',lw=2,ls='--')
    plt.xlim([-0.01,1.0])
    plt.ylim([0.0,1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('The area under the ROC curve\n')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
#朴素贝叶斯

from sklearn.naive_bayes import GaussianNB

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import QuantileTransformer#将特征进行放缩，就是将数据映射到一个分布中去

In [ ]:
train.set_index('ID_code',inplace=True)

In [ ]:
transoformed=pd.DataFrame(QuantileTransformer(output_distribution='normal').fit_transform(train))

pipeline=make_pipeline(QuantileTransformer(output_distribution='normal'),GaussianNB())
pipeline.fit(train,target)

In [ ]:
cross_val_score(pipeline,train,target,scoring='roc_auc',cv=10).mean()

In [ ]:
roc_auc(target,pipeline.predict_proba(train)[:,1],6,6)

In [ ]:
test.set_index('ID_code',inplace=True)

In [ ]:
pred_bayes=pipeline.predict_proba(test)[:,1]

sns.set()
plt.hist(pred_bayes,bins=50)
plt.gca().set_facecolor('lightblue')
min(pred_bayes),max(pred_bayes)

In [ ]:
#使用PCA对特征进行降维

from sklearn.decomposition import PCA
pca=PCA(n_components=2)
train_2d=pca.fit_transform(train)
test_2d=pca.fit_transform(test)

train_2d.shape,test_2d.shape

In [ ]:
def plot_task(X,y,headline):
    colors=['darkcyan','red']
    classes=['Class:0','Class:1']
    
    plt.style.use('seaborn-whitegrid')
    plt.figure(figsize=(7,7),facecolor='lightyellow')
    
    for u,c in zip(np.unique(y),colors):
        plt.scatter(X[y==u,0],X[y==u,1],c=c,s=0.3)
    
    plt.gca()
    plt.title(headline,fontsize=14)
    plt.legend(classes,loc=2)
    plt.show()

In [ ]:
plot_task(train_2d,target,'Scatter graph for TrainSet_2D\n')

In [ ]:
train_2d[target==0,1]

**使用Light Gradient-Boosting Machine**

In [ ]:
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.model_selection import cross_val_score,StratifiedKFold,train_test_split

**对超参数进行优化**

In [ ]:
def lgbm_cl_bo(min_child_samples,colsample_bytree,num_leaves,learning_rate):
    
    params_lgbm={}
    params_lgbm['min_child_samples']=round(min_child_samples)
    params_lgbm['colsample_bytree']=colsample_bytree
    params_lgbm['num_leaves']=round(num_leaves)
    params_lgbm['learning_rate']=learning_rate
    params_lgbm['boosting_type']='gbdt'
    params_lgbm['objective']='binary'
    params_lgbm['subsample']=1.0
    params_lgbm['max_bin']=1023
    params_lgbm['n_jobs']=-1
    params_lgbm['reg_alpha']=0.1
    params_lgbm['reg_lambda']=0.2
    params_lgbm['n_estimators']=1200
    
    score=cross_val_score(LGBMClassifier(**params_lgbm,random_state=42),train,target,scoring='roc_auc',
                         cv=5).mean()
    
    score=score.mean()
    return score

In [ ]:
params_lgbm={'min_child_samples':(800,1200),
            'colsample_bytree':(0.3,1),
            'num_leaves':(20,60),
            'learning_rate':(0.005,0.06)}
lgbm_bo=BayesianOptimization(lgbm_cl_bo,params_lgbm,random_state=42)
lgbm_bo.maximize(n_iter=5,init_points=5)